In [ ]:
import contextlib

In [ ]:
@contextlib.contextmanager
def context():
    print('start')
    print('a')
    try:
        yield "abc"
    finally:
        print("exit")

In [ ]:
with context():
    print('haha')

In [ ]:
type(context)

In [ ]:
def add(x,y):
    yield x+y

In [ ]:
add(1,2).__name__

In [ ]:
class ContextManager:
    def __init__(self,fn):
        self.fn = fn
    def __call__(self,*args,**kwargs):
        self.gen = self.fn(*args,**kwargs)
        return self 
    def __enter__(self):
        try:
            return next(self.gen)
        except StopIteration:
            pass
    def __exit__(self,exc_type,exc,traceback):
        try:
            return next(self.gen)
        except StopIteration:
            pass
        return False

In [ ]:
@ContextManager
def context2(x,y):
    print('start')
    print('a')
    try:
        yield x+y
    finally:
        print("exit")

In [ ]:
with context2(1,2):
    print('haha')

In [1]:
import threading

In [2]:
def worker():
    print('work')

In [3]:
thread = threading.Thread(target=worker)

In [4]:
thread.start()

work


In [4]:
import threading
import logging
import importlib
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')
class Task:
    def __init__(self,name):
        self.name = name
        self.lock = threading.Lock()
def worker(tasks):
    for task in tasks:
        if task.lock.acquire(False):
            logging.info(task.name)
tasks = [Task(x) for x in range(10)]
for x in range(5):
    threading.Thread(target=worker,args=(tasks,),name='worker-{}'.format(x))

In [5]:
logging.info('aaa')

2017-03-11 16:58:03,901 INFO [MainThread] aaa


In [17]:
import random
class Dispatcher:
    def __init__(self):
        self.data = None
        self.event = threading.Event()
        self.cond = threading.Condition()
    
    def consumer(self):
        while not self.event.is_set():
            with self.cond:
                self.cond.wait()
                logging.info(self.data)
    def producer(self):
        for _ in range(10):
            data = random.randint(0,100)
            logging.info(data)
            self.data = data
            with self.cond:
                self.cond.notify_all()
            self.event.wait(1)
        self.event.set()

In [18]:
d = Dispatcher()

In [19]:
c = threading.Thread(target=d.consumer,name='consumer')

In [20]:
p = threading.Thread(target=d.producer,name='producer')

In [21]:
c.start()

In [22]:
p.start()

2017-03-11 17:32:44,784 INFO [producer] 20
2017-03-11 17:32:44,786 INFO [consumer] 20
2017-03-11 17:32:45,787 INFO [producer] 18
2017-03-11 17:32:45,789 INFO [consumer] 18
2017-03-11 17:32:46,789 INFO [producer] 26
2017-03-11 17:32:46,791 INFO [consumer] 26
2017-03-11 17:32:47,791 INFO [producer] 83
2017-03-11 17:32:47,809 INFO [consumer] 83
2017-03-11 17:32:48,825 INFO [producer] 15
2017-03-11 17:32:48,830 INFO [consumer] 15


In [1]:
import threading

In [2]:
import logging
import importlib

In [3]:
importlib.reload(logging)

<module 'logging' from '/root/.pyenv/versions/3.5.2/lib/python3.5/logging/__init__.py'>

In [4]:
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')

In [5]:
logging.info('haah')

2017-03-11 22:32:29,829 INFO [MainThread] haah


In [7]:
def worker(n):
    logging.info(n)

In [14]:
t = threading.Thread(target=worker,args=(3,),name='worker')

In [17]:
t.start()

2017-03-11 22:44:38,877 INFO [worker] 3


In [18]:
t.ident

140643510765312

In [9]:
threading.Thread(target=worker,args=(3,)).start()

2017-03-11 22:42:54,748 INFO [Thread-4] 3


In [2]:
import threading
import logging
import importlib
importlib.reload(logging)
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(levelname)s [%(threadName)s] %(message)s')


In [2]:
import random
import datetime
def worker(event: threading.Event):
    s = random.randint(1, 5)
    #time.sleep(s)
    event.wait(s)
    event.set()         #set()调用，把所以在wait()的线程都 唤醒执行。
    logging.info('sleep {}'.format(s))
def boss(event: threading.Event):
    start = datetime.datetime.now()
    event.wait()         #主线程阻塞等待直到发生set()调用
    logging.info('worker exit {}'.format(datetime.datetime.now() - start))
event = threading.Event()
def start():
    event = threading.Event()
    b = threading.Thread(target=boss, args=(event, ), name='boss')
    b.start()
    for x in range(5):
        threading.Thread(target=worker, args=(event, ), name='worker-{}'.format(x)).start()

In [3]:
start()

2017-03-13 10:05:39,989 INFO [worker-1] sleep 1
2017-03-13 10:05:39,990 INFO [boss] worker exit 0:00:01.001747
2017-03-13 10:05:39,990 INFO [worker-0] sleep 3
2017-03-13 10:05:39,990 INFO [worker-2] sleep 5
2017-03-13 10:05:39,990 INFO [worker-3] sleep 1
2017-03-13 10:05:39,999 INFO [worker-4] sleep 4


In [5]:
import time
class Task:
    def __init__(self,name):
        self.name = name
        self.lock = threading.Lock()
tasks = [Task(x) for x in range(10)]
def worker(tasks):
    for task in tasks:
        if task.lock.acquire(False):
            logging.info(task.name)
            time.sleep(1)
for x in range(5):
    threading.Thread(target=worker,args=(tasks,),name='worker-{}'.format(x)).start()

2017-03-13 10:23:52,474 INFO [worker-0] 0
2017-03-13 10:23:52,477 INFO [worker-1] 1
2017-03-13 10:23:52,478 INFO [worker-2] 2
2017-03-13 10:23:52,480 INFO [worker-3] 3
2017-03-13 10:23:52,482 INFO [worker-4] 4
2017-03-13 10:23:53,479 INFO [worker-0] 5
2017-03-13 10:23:53,479 INFO [worker-1] 6
2017-03-13 10:23:53,483 INFO [worker-2] 7
2017-03-13 10:23:53,484 INFO [worker-3] 8
2017-03-13 10:23:53,486 INFO [worker-4] 9


In [13]:
data = 0
def worker(n):
    global data
    data += n
    data -= n
    
def change(v):
    for _ in range(100):
        worker(v)

    
t1 =threading.Thread(target=change,args=(10,))
t2 = threading.Thread(target=change,args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(data)

0


In [14]:
lock = threading.Lock()

In [15]:
help(lock.acquire)

Help on built-in function acquire:

acquire(...) method of _thread.lock instance
    acquire(blocking=True, timeout=-1) -> bool
    (acquire_lock() is an obsolete synonym)
    
    Lock the lock.  Without argument, this blocks if the lock is already
    locked (even by the same thread), waiting for another thread to release
    the lock, and return True once the lock is acquired.
    With an argument, this will only block if the argument is true,
    and the return value reflects whether the lock is acquired.
    The blocking operation is interruptible.



In [16]:
lock.acquire(blocking=False)

True

In [17]:
lock.acquire()

KeyboardInterrupt: 

In [ ]:
lock.acquire(blocking=False)

In [ ]:
lock.release()

In [ ]:
lokc.release()

In [18]:
Exception.__mro__

(Exception, BaseException, object)

In [19]:
KeyboardInterrupt.__mro__

(KeyboardInterrupt, BaseException, object)

In [20]:
SystemExit.__mro__

(SystemExit, BaseException, object)

In [21]:
GeneratorExit.__mro__

(GeneratorExit, BaseException, object)

In [1]:
%ls

15-22.log          du.py         nain.py       t2.py
aaa.ipynb          find.py       new/          test.py
app.py             funselect.py  Njz.py        text
argparseme.py      gwc.py        nn/           topheap.py
a.txt              hanoi.py      npoland.py    url.py
cat.py             juzhen.py     print.py      watch.py
cda.py             leixing.py    __pycache__/  wrap.py
ContextManager.py  LinkList.py   Qpoland.py    yh.py
cp.py              log/          README.md     zuoye/
dist/              ls.py         stackfun.py   作业 记录.ipynb
DoubleLinkList.py  ls.spec       t1.txt


In [5]:
dic = {}
with open('./15-22.log') as f:
    for i in f.readlines():
        key = i.split('||')[4]
        dic[key] = i 

In [157]:
def filetodic(x):
    dic = {}
    def StrtoInt(x):
        if x == '' or x == '\n':
            return 0
        new = x.strip(' %\n\t\r')
        for i in new:
            if i not in '0123456789.' or x =='':
                return new
        return float(new)
    with open(x) as f:
        for i in f.readlines():
            sp = i.split('||') 
            dic[i] = (int(sp[3].split(':')[-1].strip('%')),StrtoInt(sp[4].split(':')[-1]),StrtoInt(sp[5].split(':')[-1]),StrtoInt(sp[6].split(':')[-1]))
    return sorted(dic,key=lambda x:(dic[x][0],dic[x][1]))


In [158]:
dic = filetodic('./15-22.log')

In [118]:
def StrtoInt(x):
        lst = []
        new = x.strip(' %\n\t\r')
        if x == '':
            return x
        print(new)
        for i in new:
            if i not in '0123456789.' or x =='':
                return new
        lst.append(new)
        print(lst)
        return float(new)

In [119]:
StrtoInt('')

''

In [87]:
'0%'.strip(' %')

'0'

In [84]:
float('100')

100.0

In [78]:
for i in '12.3':
    if i not in '0123456789.':
        print('aaaaa')
    else:
        print(i)

1
2
.
3
